In [1]:
import pandas
import faker

In [2]:
df=pandas.read_csv("/Users/yashagarwal/Darsh/hackathon/OPEN Hack/datasets_EasyEats/finaldsgpt3.5_austin.csv")


In [3]:

'''columns = ['Proteins', 'Minerals', 'Vitamins', 'Carbohydrates', 'Fats', 'Fibers', 'Sugars']

for col in columns:
    df[col] = df[col].replace({'Low': 1, 'Medium': 2, 'High': 3})'''


"columns = ['Proteins', 'Minerals', 'Vitamins', 'Carbohydrates', 'Fats', 'Fibers', 'Sugars']\n\nfor col in columns:\n    df[col] = df[col].replace({'Low': 1, 'Medium': 2, 'High': 3})"

In [4]:
df2=pandas.DataFrame()
df2["Food Item"]=df["item_name"]

## Calculating explicit Utility for our dishes with example of sample diets

In [5]:
def calculate_explicit_utility_weightloss(proteins_rank, minerals_rank, vitamins_rank, carbohydrates_rank, fats_rank, fibers_rank, sugars_rank,calories_rank,user_calorie,goal):
    
    # Define mapping of rankings to weights
    ranking_weights = {'Low': 1, 'Medium': 2, 'High': 3}
    if goal==0:
        weights = {'Proteins': 3, 'Minerals': 2, 'Vitamins': 2, 'Carbohydrates': 1, 'Fats': 1, 'Fibers': 3, 'Sugars': 1}
        calorie=user_calorie-100
    elif goal==1:
        weights = {'Proteins': 2.5, 'Minerals': 2, 'Vitamins': 2, 'Carbohydrates': 2, 'Fats': 2, 'Fibers': 3, 'Sugars': 1}
        calorie=user_calorie
    elif goal==2:
        weights = {'Proteins': 3, 'Minerals': 2.5, 'Vitamins': 2.5, 'Carbohydrates': 2.5, 'Fats': 2.5, 'Fibers': 1.5, 'Sugars': 2}
        calorie=user_calorie+100
        

    # Calculate scores for each nutrient category
    scores = {'Proteins': weights['Proteins']-ranking_weights[proteins_rank] >0,
              'Minerals': (ranking_weights[minerals_rank] - weights['Minerals']) if -ranking_weights[minerals_rank] + weights['Minerals'] > 0 else 1.2*(ranking_weights[minerals_rank] - weights['Minerals']),
              'Vitamins': (-ranking_weights[vitamins_rank] + weights['Vitamins']) if -ranking_weights[vitamins_rank] + weights['Vitamins'] > 0 else 0.8*(-ranking_weights[vitamins_rank] + weights['Vitamins']),
              'Carbohydrates': (-ranking_weights[carbohydrates_rank] + weights['Carbohydrates']) if -ranking_weights[carbohydrates_rank] + weights['Carbohydrates'] > 0 else (-ranking_weights[carbohydrates_rank] + weights['Carbohydrates'])**2,
              'Fats': (-ranking_weights[fats_rank] + weights['Fats']) if -ranking_weights[fats_rank] + weights['Fats'] > 0 else (-ranking_weights[fats_rank] + weights['Fats'])**2,
              'Fibers': (-ranking_weights[fibers_rank] + weights['Fibers']),
              'Sugars': (-ranking_weights[sugars_rank] + weights['Sugars']) if -ranking_weights[sugars_rank] + weights['Sugars'] > 0 else (-ranking_weights[sugars_rank] + weights['Sugars'])**2,
              'calories': (-calories_rank + calorie)/100 if -calories_rank + calorie > 0 else (-calories_rank/100 + calorie/100)*-0.8}
    utility_score = sum(scores.values())
    return utility_score

explicit_dietry_utility=[]

for i in range(0, len(df)):
    print(df.iloc[i]['Proteins'], df.iloc[i]['Minerals'], df.iloc[i]['Vitamins'], df.iloc[i]['Carbohydrates'], df.iloc[i]['Fats'], df.iloc[i]['Fibers'], df.iloc[i]['Sugars'],df.iloc[i]['Calories'])
    utility_score = calculate_explicit_utility_weightloss(df.iloc[i]['Proteins'], df.iloc[i]['Minerals'], df.iloc[i]['Vitamins'], df.iloc[i]['Carbohydrates'], df.iloc[i]['Fats'], df.iloc[i]['Fibers'], df.iloc[i]['Sugars'],df.iloc[i]['Calories'],600,0)
    print("Utility Score:", utility_score)
    explicit_dietry_utility.append(utility_score)
    print("\n")

df2['Explicit Dietry Utility'] = explicit_dietry_utility


High Medium Low Medium Low Low Low 240.0
Utility Score: 6.6


High Medium Low High Medium Low Low 800.0
Utility Score: 10.4


Medium Low Medium Medium Low High Low 250.0
Utility Score: 3.5


High Medium Medium Medium Medium Low Low 350.0
Utility Score: 5.5


High Medium Low Medium Medium Low Low 250.0
Utility Score: 7.5


High Medium Low Medium Medium Low Low 250.0
Utility Score: 7.5


High Medium Medium Low High Medium Low 300.0
Utility Score: 7.0


Medium Low High Medium Low Medium Low 180.0
Utility Score: 4.4


High Medium Low Low Medium Low Low 250.0
Utility Score: 6.5


High Medium Low High High Low Low 450.0
Utility Score: 11.5


Medium Low Low Medium High Low Medium 350.0
Utility Score: 10.5


High Medium Low Low Medium Low Low 250.0
Utility Score: 6.5


Medium Low Low Medium Low Low Low 250.0
Utility Score: 6.5


High Medium Medium Low High Low Low 600.0
Utility Score: 6.8


High Medium Low Low Medium Low Low 300.0
Utility Score: 6.0


High Medium Low Medium Medium Low Low 300.

In [6]:
df2

,Food Item,Explicit Dietry Utility
0,Spicy Tuna Roll,6.6
1,2 Chicken Chalupas Supreme Combo,10.4
2,California Roll,3.5
3,Philadelphia Roll,5.5
4,Gyoza,7.5
...,...,...
2508,Power Crunch Protein Bar Peanut Butter Cream -...,9.0
2509,Almased Low-Glycemic High-Protein Formula Natu...,2.7
2510,Nature's Way Alive! Men's Gummies Vitamins Fru...,10.2
2511,Mommy's Bliss Baby Probiotic Drops Everyday 45...,6.1


## Using price and ratings as part of explict implicit utility

In [7]:
#explicit implicit utility
ratings_utility = []
print(len(df))
for i in range(0,len(df)):
    ratings_utility.append(5-df["score"].iloc[i])
df2["Ratings Utility"] = ratings_utility


price_utility=[]
for i in range(0,len(df)):
    price_utility.append(float(df["price"].iloc[i][:-4])/10)
df2["Price Utility"] = price_utility



2513


In [8]:
df2

,Food Item,Explicit Dietry Utility,Ratings Utility,Price Utility
0,Spicy Tuna Roll,6.6,0.3,0.950
1,2 Chicken Chalupas Supreme Combo,10.4,0.4,1.235
2,California Roll,3.5,0.3,0.750
3,Philadelphia Roll,5.5,0.3,0.950
4,Gyoza,7.5,0.3,0.750
...,...,...,...,...
2508,Power Crunch Protein Bar Peanut Butter Cream -...,9.0,0.2,0.208
2509,Almased Low-Glycemic High-Protein Formula Natu...,2.7,0.2,3.148
2510,Nature's Way Alive! Men's Gummies Vitamins Fru...,10.2,0.2,1.416
2511,Mommy's Bliss Baby Probiotic Drops Everyday 45...,6.1,0.2,3.043


In [9]:
df2["Primary Flavour"]=df["Primary_Flavour"]
df2["Secondary Flavour"]=df["Secondary_Flavour"]
df2["Dish_type"]=df["Dish_type"]


### Sampling a set to hold placeholder for view item by users.

In [10]:
import random
ts = [round(random.uniform(1, 4), 1) for _ in range(len(df2))]

df2["Time spent"]=ts
tsma=df2["Time spent"].max()
tsmi=df2["Time spent"].min()
print()
tsu=[]
for i in range(0,len(df2)):
    tsu.append((round((df2["Time spent"].iloc[i]-tsmi)/(tsma-tsmi),2)))
df2["Time Spent Utility"]=tsu


In [11]:
df2
df2['restaurant_name']=df['restaurant_name']


In [12]:
df2["Allergens"]=df["Alergens"]

In [13]:
sampled_df = df2.sample(n=50)
sampled_df


,Food Item,Explicit Dietry Utility,Ratings Utility,Price Utility,Primary Flavour,Secondary Flavour,Dish_type,Time spent,Time Spent Utility,restaurant_name,Allergens
2184,Kids' Boneless Wings,9.60,0.8,0.699,Savory,Creamy,appetizers,2.2,0.40,Buffalo Wild Wings (13000 N Interstate 35 Bldg 4),NaN
1964,The Southern Charm Burger®,14.00,0.4,1.689,Sweet,Savory,main course,2.2,0.40,Red Robin Gourmet Burgers (13000 N IH35 SBFR #5),NaN
1712,Rodeo Ring Beyond Burger®,8.92,0.3,1.413,Savory,Rich,main course,1.4,0.13,The Burger Den (1-35 North Parmer Lane),"['Cheese', 'White Cheddar cheese', 'Beer-Batte..."
1492,Crazy Yellowtail Roll,3.50,0.6,1.599,Spicy,Umami,appetizers,1.9,0.30,Kublai Khan,NaN
455,"Bacon, Egg and Cheddar Breakfast Burrito",11.20,0.6,1.125,Savory,Rich,main course,1.4,0.13,Burro's Breakfast Burritos (13000 N Interstate...,NaN
2099,Pepper Pals® Pepperoni Pizza,11.40,0.5,0.655,Savory,Rich,main course,2.3,0.43,Chili's (12901 N IH 35),"['Gluten', 'Dairy', 'Soy']"
1185,Classic Chicken Tikka Triumph,9.60,1.0,1.429,Spicy,Rich,main course,1.6,0.20,Curry Consortium (500 W Canyon Ridge Dr Suite ...,NaN
1166,Shrimp w Walnut,6.00,0.2,2.270,Savory,Nutty,appetizers,1.5,0.17,Ho Ho Chinese BBQ 好好烧腊,NaN
204,Chicken Shawerma,3.80,0.3,0.999,Savory,Spicy,main course,2.1,0.37,Peace Bakery and Deli (N Lamar Blvd),"['Garlic', 'Tomatoes', 'Pickles']"
2149,Power Menu Bowl - Veggie,6.40,0.4,0.719,Savory,Creamy,main course,3.5,0.83,Taco Bell (1548 E. Parmer Lane),NaN


In [14]:
# user viewed items
freq_list=sampled_df["Primary Flavour"].value_counts()
freq_list.to_dict()



{'Savory': 22,
 'Sweet': 12,
 'Spicy': 11,
 'Umami': 2,
 'Refreshing': 2,
 'Rich': 1}

## IU- Implicit Utility calculation from the first implicit MAU (Multiple attribute utility)

In [15]:
#implicit utility of primary flavour
freq_utility={}
for k in freq_list.keys():
    freq_utility[k]=round((freq_list[k]-min(freq_list.values))/(max(freq_list.values)-min(freq_list.values)),2)
freq_utility
freq_utility2={}


In [16]:
freq_list2=sampled_df["Secondary Flavour"].value_counts()
freq_list2.to_dict()



{'Rich': 12,
 'Umami': 9,
 'Savory': 7,
 'Creamy': 6,
 'Nutty': 6,
 'Cheesy': 3,
 'Spicy': 2,
 'Refreshing': 2,
 'Tangy': 2,
 'Salty': 1}

In [17]:
freq_utility2={}
for k in freq_list2.keys():
    freq_utility2[k]=round((freq_list2[k]-min(freq_list2.values))/(max(freq_list2.values)-min(freq_list2.values)),2)
freq_utility2


{'Rich': 1.0,
 'Umami': 0.73,
 'Savory': 0.55,
 'Creamy': 0.45,
 'Nutty': 0.45,
 'Cheesy': 0.18,
 'Spicy': 0.09,
 'Refreshing': 0.09,
 'Tangy': 0.09,
 'Salty': 0.0}

In [18]:
iu1=[]
iu2=[]
for i in range(0,len(df2)):
    if df2["Primary Flavour"].iloc[i] in freq_utility.keys():
        iu1.append(freq_utility[df2["Primary Flavour"].iloc[i]])
    else:
        iu1.append(0)
    if df2["Secondary Flavour"].iloc[i] in freq_utility2.keys():
        iu2.append(freq_utility2[df2["Secondary Flavour"].iloc[i]])
    else:
        iu2.append(0)
df2["Implicit Utility 1"]=iu1
df2["Implicit Utility 2"]=iu2
df2

,Food Item,Explicit Dietry Utility,Ratings Utility,Price Utility,Primary Flavour,Secondary Flavour,Dish_type,Time spent,Time Spent Utility,restaurant_name,Allergens,Implicit Utility 1,Implicit Utility 2
0,Spicy Tuna Roll,6.6,0.3,0.950,Spicy,Umami,appetizers,1.8,0.27,Tsukimi,"['Fish', 'Soy', 'Wheat']",0.48,0.73
1,2 Chicken Chalupas Supreme Combo,10.4,0.4,1.235,Savory,Spicy,main course,4.0,1.00,Taco Bell (1548 E. Parmer Lane),NaN,1.00,0.09
2,California Roll,3.5,0.3,0.750,Savory,Refreshing,appetizers,2.5,0.50,Tsukimi,NaN,1.00,0.09
3,Philadelphia Roll,5.5,0.3,0.950,Creamy,Umami,appetizers,3.7,0.90,Tsukimi,"['salmon', 'dairy']",0.00,0.73
4,Gyoza,7.5,0.3,0.750,Savory,Umami,appetizers,1.4,0.13,Tsukimi,NaN,1.00,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,Power Crunch Protein Bar Peanut Butter Cream -...,9.0,0.2,0.208,Sweet,Nutty,dessert,1.6,0.20,Walgreens (12601 Tech Ridge Blvd),"['dairy', 'peanuts', 'whey']",0.52,0.45
2509,Almased Low-Glycemic High-Protein Formula Natu...,2.7,0.2,3.148,Sweet,Creamy,drinks,1.4,0.13,Walgreens (12601 Tech Ridge Blvd),[],0.52,0.45
2510,Nature's Way Alive! Men's Gummies Vitamins Fru...,10.2,0.2,1.416,Sweet,Refreshing,drinks,3.6,0.87,Walgreens (12601 Tech Ridge Blvd),"['dairy', 'yeast', 'wheat', 'peanuts', 'eggs',...",0.52,0.09
2511,Mommy's Bliss Baby Probiotic Drops Everyday 45...,6.1,0.2,3.043,Sweet,Refreshing,condiment,2.1,0.37,Walgreens (12601 Tech Ridge Blvd),NaN,0.52,0.09


In [19]:
sampled_df = pandas.merge(sampled_df, df2, on='Food Item', how='inner')



In [20]:
sampled_df

,Food Item,Explicit Dietry Utility_x,Ratings Utility_x,Price Utility_x,Primary Flavour_x,Secondary Flavour_x,Dish_type_x,Time spent_x,Time Spent Utility_x,restaurant_name_x,...,Price Utility_y,Primary Flavour_y,Secondary Flavour_y,Dish_type_y,Time spent_y,Time Spent Utility_y,restaurant_name_y,Allergens_y,Implicit Utility 1,Implicit Utility 2
0,Kids' Boneless Wings,9.60,0.8,0.699,Savory,Creamy,appetizers,2.2,0.40,Buffalo Wild Wings (13000 N Interstate 35 Bldg 4),...,0.699,Savory,Creamy,appetizers,2.2,0.40,Buffalo Wild Wings (13000 N Interstate 35 Bldg 4),NaN,1.00,0.45
1,The Southern Charm Burger®,14.00,0.4,1.689,Sweet,Savory,main course,2.2,0.40,Red Robin Gourmet Burgers (13000 N IH35 SBFR #5),...,1.689,Sweet,Savory,main course,2.2,0.40,Red Robin Gourmet Burgers (13000 N IH35 SBFR #5),NaN,0.52,0.55
2,Rodeo Ring Beyond Burger®,8.92,0.3,1.413,Savory,Rich,main course,1.4,0.13,The Burger Den (1-35 North Parmer Lane),...,1.413,Savory,Rich,main course,1.4,0.13,The Burger Den (1-35 North Parmer Lane),"['Cheese', 'White Cheddar cheese', 'Beer-Batte...",1.00,1.00
3,Crazy Yellowtail Roll,3.50,0.6,1.599,Spicy,Umami,appetizers,1.9,0.30,Kublai Khan,...,1.599,Spicy,Umami,appetizers,1.9,0.30,Kublai Khan,NaN,0.48,0.73
4,"Bacon, Egg and Cheddar Breakfast Burrito",11.20,0.6,1.125,Savory,Rich,main course,1.4,0.13,Burro's Breakfast Burritos (13000 N Interstate...,...,1.125,Savory,Rich,main course,1.4,0.13,Burro's Breakfast Burritos (13000 N Interstate...,NaN,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,KADAI BIRYANI,9.40,0.3,1.500,Spicy,Savory,main course,2.5,0.50,Masala Wok (Tech Ridge),...,1.500,Spicy,Savory,main course,2.5,0.50,Masala Wok (Tech Ridge),"['Eggs', 'Dairy', 'Gluten']",0.48,0.55
57,Banh Xeo,10.50,0.7,1.595,Savory,Umami,main course,3.8,0.93,Cyclo Vietnamese and Chinese Cuisine,...,1.595,Savory,Umami,main course,3.8,0.93,Cyclo Vietnamese and Chinese Cuisine,NaN,1.00,0.73
58,Orange County Roll,5.00,0.3,0.950,Refreshing,Umami,appetizers,2.5,0.50,Tsukimi,...,0.950,Refreshing,Umami,appetizers,2.5,0.50,Tsukimi,NaN,0.05,0.73
59,Homemade Chicken Noodle,1.70,0.5,0.325,Savory,Umami,main course,1.4,0.13,Wings N More Austin,...,0.325,Savory,Umami,main course,1.4,0.13,Wings N More Austin,"['Chicken', 'Noodles', 'Vegetables']",1.00,0.73


In [21]:
sampled_df1=sampled_df.iloc[:, 0:12]

In [22]:
sampled_df1

,Food Item,Explicit Dietry Utility_x,Ratings Utility_x,Price Utility_x,Primary Flavour_x,Secondary Flavour_x,Dish_type_x,Time spent_x,Time Spent Utility_x,restaurant_name_x,Allergens_x,Explicit Dietry Utility_y
0,Kids' Boneless Wings,9.60,0.8,0.699,Savory,Creamy,appetizers,2.2,0.40,Buffalo Wild Wings (13000 N Interstate 35 Bldg 4),NaN,9.60
1,The Southern Charm Burger®,14.00,0.4,1.689,Sweet,Savory,main course,2.2,0.40,Red Robin Gourmet Burgers (13000 N IH35 SBFR #5),NaN,14.00
2,Rodeo Ring Beyond Burger®,8.92,0.3,1.413,Savory,Rich,main course,1.4,0.13,The Burger Den (1-35 North Parmer Lane),"['Cheese', 'White Cheddar cheese', 'Beer-Batte...",8.92
3,Crazy Yellowtail Roll,3.50,0.6,1.599,Spicy,Umami,appetizers,1.9,0.30,Kublai Khan,NaN,3.50
4,"Bacon, Egg and Cheddar Breakfast Burrito",11.20,0.6,1.125,Savory,Rich,main course,1.4,0.13,Burro's Breakfast Burritos (13000 N Interstate...,NaN,11.20
...,...,...,...,...,...,...,...,...,...,...,...,...
56,KADAI BIRYANI,9.40,0.3,1.500,Spicy,Savory,main course,2.5,0.50,Masala Wok (Tech Ridge),"['Eggs', 'Dairy', 'Gluten']",9.40
57,Banh Xeo,10.50,0.7,1.595,Savory,Umami,main course,3.8,0.93,Cyclo Vietnamese and Chinese Cuisine,NaN,10.50
58,Orange County Roll,5.00,0.3,0.950,Refreshing,Umami,appetizers,2.5,0.50,Tsukimi,NaN,5.00
59,Homemade Chicken Noodle,1.70,0.5,0.325,Savory,Umami,main course,1.4,0.13,Wings N More Austin,"['Chicken', 'Noodles', 'Vegetables']",1.70


## Non linear optitimization of Implicity Utility (Flavour)

In [23]:

import numpy as np
from scipy.optimize import minimize
def objective(weights):
    fs = 0
    for i in range(len(df2)):
        s = 0
        for j in range(2):
            if j == 0:
                s += (df2["Time Spent Utility"].iloc[i] - df2["Implicit Utility 1"].iloc[i] * weights[j]) ** 2
            elif j == 1:
                s += (df2["Time Spent Utility"].iloc[i] - df2["Implicit Utility 2"].iloc[i] * weights[j]) ** 2
        fs += s
    return fs


In [24]:
def constraint(weights):
    return 1 - np.sum(weights)

In [25]:
w0=[0.5,0.5]
objective(w0)

785.0099250000036

### Using Sequentential Least square Programming (SLSQP) to solve the problem

In [26]:

bound=(0,1)
bnd=[bound,bound]
con={"type":"eq","fun":constraint}
sol=minimize(objective,w0,method="SLSQP",bounds=bnd,constraints=con)
weights=sol.x.tolist()
print(weights)
print(objective(weights))


[0.4834121981449204, 0.5165878018550796]
784.3654741335092


## Calculating final utility for the dishes

In [27]:
df2

,Food Item,Explicit Dietry Utility,Ratings Utility,Price Utility,Primary Flavour,Secondary Flavour,Dish_type,Time spent,Time Spent Utility,restaurant_name,Allergens,Implicit Utility 1,Implicit Utility 2
0,Spicy Tuna Roll,6.6,0.3,0.950,Spicy,Umami,appetizers,1.8,0.27,Tsukimi,"['Fish', 'Soy', 'Wheat']",0.48,0.73
1,2 Chicken Chalupas Supreme Combo,10.4,0.4,1.235,Savory,Spicy,main course,4.0,1.00,Taco Bell (1548 E. Parmer Lane),NaN,1.00,0.09
2,California Roll,3.5,0.3,0.750,Savory,Refreshing,appetizers,2.5,0.50,Tsukimi,NaN,1.00,0.09
3,Philadelphia Roll,5.5,0.3,0.950,Creamy,Umami,appetizers,3.7,0.90,Tsukimi,"['salmon', 'dairy']",0.00,0.73
4,Gyoza,7.5,0.3,0.750,Savory,Umami,appetizers,1.4,0.13,Tsukimi,NaN,1.00,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,Power Crunch Protein Bar Peanut Butter Cream -...,9.0,0.2,0.208,Sweet,Nutty,dessert,1.6,0.20,Walgreens (12601 Tech Ridge Blvd),"['dairy', 'peanuts', 'whey']",0.52,0.45
2509,Almased Low-Glycemic High-Protein Formula Natu...,2.7,0.2,3.148,Sweet,Creamy,drinks,1.4,0.13,Walgreens (12601 Tech Ridge Blvd),[],0.52,0.45
2510,Nature's Way Alive! Men's Gummies Vitamins Fru...,10.2,0.2,1.416,Sweet,Refreshing,drinks,3.6,0.87,Walgreens (12601 Tech Ridge Blvd),"['dairy', 'yeast', 'wheat', 'peanuts', 'eggs',...",0.52,0.09
2511,Mommy's Bliss Baby Probiotic Drops Everyday 45...,6.1,0.2,3.043,Sweet,Refreshing,condiment,2.1,0.37,Walgreens (12601 Tech Ridge Blvd),NaN,0.52,0.09


In [28]:
df2["Final Implicit utility"]=1-(df2["Implicit Utility 1"]*weights[0]+df2["Implicit Utility 2"]*weights[1])
df2

,Food Item,Explicit Dietry Utility,Ratings Utility,Price Utility,Primary Flavour,Secondary Flavour,Dish_type,Time spent,Time Spent Utility,restaurant_name,Allergens,Implicit Utility 1,Implicit Utility 2,Final Implicit utility
0,Spicy Tuna Roll,6.6,0.3,0.950,Spicy,Umami,appetizers,1.8,0.27,Tsukimi,"['Fish', 'Soy', 'Wheat']",0.48,0.73,0.390853
1,2 Chicken Chalupas Supreme Combo,10.4,0.4,1.235,Savory,Spicy,main course,4.0,1.00,Taco Bell (1548 E. Parmer Lane),NaN,1.00,0.09,0.470095
2,California Roll,3.5,0.3,0.750,Savory,Refreshing,appetizers,2.5,0.50,Tsukimi,NaN,1.00,0.09,0.470095
3,Philadelphia Roll,5.5,0.3,0.950,Creamy,Umami,appetizers,3.7,0.90,Tsukimi,"['salmon', 'dairy']",0.00,0.73,0.622891
4,Gyoza,7.5,0.3,0.750,Savory,Umami,appetizers,1.4,0.13,Tsukimi,NaN,1.00,0.73,0.139479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,Power Crunch Protein Bar Peanut Butter Cream -...,9.0,0.2,0.208,Sweet,Nutty,dessert,1.6,0.20,Walgreens (12601 Tech Ridge Blvd),"['dairy', 'peanuts', 'whey']",0.52,0.45,0.516161
2509,Almased Low-Glycemic High-Protein Formula Natu...,2.7,0.2,3.148,Sweet,Creamy,drinks,1.4,0.13,Walgreens (12601 Tech Ridge Blvd),[],0.52,0.45,0.516161
2510,Nature's Way Alive! Men's Gummies Vitamins Fru...,10.2,0.2,1.416,Sweet,Refreshing,drinks,3.6,0.87,Walgreens (12601 Tech Ridge Blvd),"['dairy', 'yeast', 'wheat', 'peanuts', 'eggs',...",0.52,0.09,0.702133
2511,Mommy's Bliss Baby Probiotic Drops Everyday 45...,6.1,0.2,3.043,Sweet,Refreshing,condiment,2.1,0.37,Walgreens (12601 Tech Ridge Blvd),NaN,0.52,0.09,0.702133


In [29]:
import streamlit as st

In [30]:
df2["Final Utility"]=round(df2["Explicit Dietry Utility"]/8)+df2["Ratings Utility"]+df2["Price Utility"]+df2["Final Implicit utility"]
df2.sort_values(by="Final Utility",ascending=True,inplace=True)
df2.head()

,Food Item,Explicit Dietry Utility,Ratings Utility,Price Utility,Primary Flavour,Secondary Flavour,Dish_type,Time spent,Time Spent Utility,restaurant_name,Allergens,Implicit Utility 1,Implicit Utility 2,Final Implicit utility,Final Utility
495,Chicken Caesar Salad,3.50,0.2,0.549,Savory,Rich,salads,1.5,0.17,7-Eleven (1625 E Parmer Ln),"['Dairy', 'Gluten']",1.0,1.00,0.000000,0.749000
967,Edamame,3.43,0.1,0.631,Savory,Umami,appetizers,3.8,0.93,Save the Fish Vegetarian Sushi (12901 N. I-35),['Soy'],1.0,0.73,0.139479,0.870479
1061,Slim-N-Trim™ Veggie,2.30,0.1,0.712,Savory,Umami,main course,3.7,0.90,"Smoothie King (12314 N. Interstate 35, Unit 100)","['milk', 'egg']",1.0,0.73,0.139479,0.951479
663,Homemade Chicken Noodle,1.70,0.5,0.325,Savory,Umami,main course,1.4,0.13,Wings N More Austin,"['Chicken', 'Noodles', 'Vegetables']",1.0,0.73,0.139479,0.964479
956,Taco Liflower,3.20,0.2,0.495,Savory,Creamy,main course,3.6,0.87,One Taco (Tech Ridge),NaN,1.0,0.45,0.284123,0.979123
